In [86]:
#Importamos lo necesario para trabajar con la API
import requests
import pandas as pd
import numpy as np
#para transformar texto a diccionario
import ast
import json
pd.set_option('display.max_columns', None)
pd.set_option("display.max_colwidth", None)

In [87]:
#Definimos la URL de la API como el end point
url = "https://datos.madrid.es/egob/catalogo/300107-0-agenda-actividades-eventos.json"

In [88]:
#probamos la conexión con la API
response = requests.get(url)
response.status_code

200

In [89]:
#Si la conexión es correcta, la respuesta será 200 por lo que vamos a hacer un código defensivo para que si la respuesta no es 200, nos muestre un mensaje de error
response = requests.get(url)
if response.status_code == 200:
    print("Conexión correcta")
else:
    print("Error en la conexión") 

Conexión correcta


In [90]:
#Analizando la Api, se ve que los datos que necistamos están dentro del diccionario con la clave "@graph". Vamos a unir el código anterior con la extracción de los datos
response = requests.get(url)
if response.status_code == 200:
    data = response.json()["@graph"]
    df = pd.DataFrame(data)
    df_eventos = df.copy()
else:
    print("Error en la conexión") 


In [91]:
#analizaré un poco el data frame
df_eventos.head()

,@id,@type,id,title,description,free,price,dtstart,dtend,time,excluded-days,recurrence,uid,link,event-location,references,relation,address,location,organization,audience
0,https://datos.madrid.es/egob/catalogo/tipo/evento/12677513-25-aniversario-revista-fragua.json,https://datos.madrid.es/egob/kos/actividades/Exposiciones,12677513,25º aniversario de la revista La Fragua,,1,,2025-02-24 00:00:00.0,2025-03-02 23:59:00.0,,,"{'days': 'MO,TU,WE,TH,FR,SA,SU', 'frequency': 'WEEKLY', 'interval': 1}",12677513,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=1ea220bed10d4910VgnVCM2000001f4a900aRCRD,Biblioteca Pública Municipal Iván de Vargas (Centro),{'@id': ''},{'@id': 'https://datos.madrid.es/egob/catalogo/tipo/entidadesyorganismos/5975465-biblioteca-publica-municipal-ivan-vargas-centro-.json'},"{'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Centro'}, 'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Centro/Barrio/Palacio', 'locality': 'MADRID', 'postal-code': '28005', 'street-address': 'CALLE SAN JUSTO 5'}}","{'latitude': 40.41410290844665, 'longitude': -3.7098591212735883}","{'organization-name': 'Biblioteca Pública Municipal Iván de Vargas (Centro)', 'accesibility': '1,6'}",NaN
1,https://datos.madrid.es/egob/catalogo/tipo/evento/12651622-3clonws.json,https://datos.madrid.es/egob/kos/actividades/ProgramacionDestacadaAgendaCultura,12651622,3CLONWS,"Un espectáculo que comienza con cajas improvisadas, un círculo trazado en el suelo y, en el centro, una mesa rodeada por tres actores. Los espectadores van entrando. ¿La función ha empezado ya? Los actores se transforman en payasos: Monsieur Lô, el payaso blanco, acompañado por los augustos Marcel y Airbus, que reinventan clásicos en un homenaje a los payasos de siempre. Pelucas, enormes zapatos, cáscaras de plátano, pasteles de crema, dátiles trucados, doma, fanfarrias, peleas a gritos, malabares, travesuras, destellos, piruetas, melancolía, oportunismo, glotonería, poder, descaro y una pizca de arrogancia… ¡todo con una vuelta de tuerca más!. Más información",0,,2025-03-21 19:00:00.0,2025-03-22 23:59:00.0,19:00,,"{'days': 'FR,SA', 'frequency': 'WEEKLY', 'interval': 1}",12651622,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=fc8d040e7c784910VgnVCM1000001d4a900aRCRD,Teatro Circo Price,{'@id': ''},{'@id': 'https://datos.madrid.es/egob/catalogo/tipo/entidadesyorganismos/184955-teatro-circo-price.json'},"{'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Arganzuela'}, 'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Arganzuela/Barrio/PalosFrontera', 'locality': 'MADRID', 'postal-code': '28012', 'street-address': 'RONDA ATOCHA 35'}}","{'latitude': 40.40596936645757, 'longitude': -3.698589986849812}","{'organization-name': 'Teatro Circo Price', 'accesibility': '1'}",NaN
2,https://datos.madrid.es/egob/catalogo/tipo/evento/12693864-60-premio-reina-sofia-pintura-escultura.json,https://datos.madrid.es/egob/kos/actividades/Exposiciones,12693864,60 Premio Reina Sofía de Pintura y Escultura,,1,,2025-02-27 00:00:00.0,2025-03-23 23:59:00.0,,,"{'days': 'MO,TU,WE,TH,FR,SA,SU', 'frequency': 'WEEKLY', 'interval': 1}",12693864,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=34c5130393e15910VgnVCM2000001f4a900aRCRD,Centro Cultural Casa de Vacas (Retiro),{'@id': ''},{'@id': 'https://datos.madrid.es/egob/catalogo/tipo/entidadesyorganismos/1945-centro-cultural-casa-vacas-retiro-.json'},"{'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Retiro'}, 'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Retiro/Barrio/LosJeronimos', 'locality': 'MADRID', 'postal-code': '28009', 'street-a

In [92]:
df_eventos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1297 entries, 0 to 1296
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   @id             1297 non-null   object
 1   @type           1186 non-null   object
 2   id              1297 non-null   object
 3   title           1297 non-null   object
 4   description     1297 non-null   object
 5   free            1297 non-null   int64 
 6   price           1297 non-null   object
 7   dtstart         1297 non-null   object
 8   dtend           1297 non-null   object
 9   time            1297 non-null   object
 10  excluded-days   1297 non-null   object
 11  recurrence      437 non-null    object
 12  uid             1297 non-null   object
 13  link            1297 non-null   object
 14  event-location  1297 non-null   object
 15  references      1297 non-null   object
 16  relation        1184 non-null   object
 17  address         1184 non-null   object
 18  location

Como los datos solo los queremos para nuestra carga en la base de datos, me quedaré solo con las columnas requeridas para la base de datos y las analizaré individualmente

In [93]:
#Eliminamos del data frame las columnas que no necesitamos
colum_no_necesarias = ["@id", "@type","description","free","price","excluded-days","recurrence", "uid","references","relation","audience"]
df_eventos = df_eventos.drop(columns = colum_no_necesarias)


In [94]:
df_eventos.head()

,id,title,dtstart,dtend,time,link,event-location,address,location,organization
0,12677513,25º aniversario de la revista La Fragua,2025-02-24 00:00:00.0,2025-03-02 23:59:00.0,,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=1ea220bed10d4910VgnVCM2000001f4a900aRCRD,Biblioteca Pública Municipal Iván de Vargas (Centro),"{'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Centro'}, 'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Centro/Barrio/Palacio', 'locality': 'MADRID', 'postal-code': '28005', 'street-address': 'CALLE SAN JUSTO 5'}}","{'latitude': 40.41410290844665, 'longitude': -3.7098591212735883}","{'organization-name': 'Biblioteca Pública Municipal Iván de Vargas (Centro)', 'accesibility': '1,6'}"
1,12651622,3CLONWS,2025-03-21 19:00:00.0,2025-03-22 23:59:00.0,19:00,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=fc8d040e7c784910VgnVCM1000001d4a900aRCRD,Teatro Circo Price,"{'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Arganzuela'}, 'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Arganzuela/Barrio/PalosFrontera', 'locality': 'MADRID', 'postal-code': '28012', 'street-address': 'RONDA ATOCHA 35'}}","{'latitude': 40.40596936645757, 'longitude': -3.698589986849812}","{'organization-name': 'Teatro Circo Price', 'accesibility': '1'}"
2,12693864,60 Premio Reina Sofía de Pintura y Escultura,2025-02-27 00:00:00.0,2025-03-23 23:59:00.0,,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=34c5130393e15910VgnVCM2000001f4a900aRCRD,Centro Cultural Casa de Vacas (Retiro),"{'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Retiro'}, 'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Retiro/Barrio/LosJeronimos', 'locality': 'MADRID', 'postal-code': '28009', 'street-address': 'PASEO COLOMBIA 1'}}","{'latitude': 40.41906571939961, 'longitude': -3.684053112760816}","{'organization-name': 'Centro Cultural Casa de Vacas (Retiro)', 'accesibility': '1'}"
3,12691220,7 pares de pies,2025-03-23 12:00:00.0,2025-03-23 23:59:00.0,12:00,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=9d8ec08889415910VgnVCM2000001f4a900aRCRD,Centro Cultural Antonio Machado (San Blas - Canillejas),"{'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/SanBlas-Canillejas'}, 'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/SanBlas-Canillejas/Barrio/Arcos', 'locality': 'MADRID', 'postal-code': '28037', 'street-address': 'CALLE SAN ROMAN DEL VALLE 8'}}","{'latitude': 40.420280457570335, 'longitude': -3.621730741880216}","{'organization-name': 'Centro Cultural Antonio Machado (San Blas - Canillejas)', 'accesibility': '1'}"
4,12518159,A la fresca,2025-02-06 00:00:00.0,2025-02-23 23:59:00.0,,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=cd729f45bf9e1910VgnVCM1000001d4a900aRCRD,Nave 10 Matadero. Creación Dramática Contemporánea,"{'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Arganzuela'}, 'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Arganzuela/Barrio/Chopera', 'locality': 'MADRID', 'postal-code': '28045', 'street-address': 'PASEO CHOPERA 14'}}","{'latitude': 40.39245798191536, 'longitude': -3.6972067902718897}","{'organization-name': 'Nave 10 Matadero. Creación Dramática Contemporánea', 'accesibility': '1,6'}"


In [95]:
df_eventos["address"]

0                                       {'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Centro'}, 'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Centro/Barrio/Palacio', 'locality': 'MADRID', 'postal-code': '28005', 'street-address': 'CALLE SAN JUSTO 5'}}
1                           {'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Arganzuela'}, 'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Arganzuela/Barrio/PalosFrontera', 'locality': 'MADRID', 'postal-code': '28012', 'street-address': 'RONDA ATOCHA 35'}}
2                                   {'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Retiro'}, 'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Retiro/Barrio/LosJeronimos', 'locality': 'MADRID', 'postal-co

In [ ]:
#Crearé una función para extraer el código postal de la dirección
def obtener_codigo_postal(address):
    try:
        # Si es un string, convertirlo a diccionario JSON
        if isinstance(address, str):
            address = json.loads(address.replace("'", "\""))  
        
        # Extraer postal-code si existe dentro de "area"
        return address.get("area", {}).get("postal-code", "No disponible") if isinstance(address, dict) else "No disponible"
    except (json.JSONDecodeError, TypeError, AttributeError):  # Si hay error en la conversión
        return "No disponible"

# Aplicar la función a la columna y crear la nueva columna "postal_code"
df_eventos["postal_code"] = df_eventos["address"].apply(obtener_codigo_postal)
df_eventos["postal_code"]



0       28005
1       28012
2       28009
3       28037
4       28045
        ...  
1292    28015
1293    28030
1294    28039
1295    28031
1296    28015
Name: postal_code, Length: 1297, dtype: object

In [101]:
df_eventos.head()

,id,title,dtstart,dtend,time,link,event-location,address,location,organization,postal_code
0,12677513,25º aniversario de la revista La Fragua,2025-02-24 00:00:00.0,2025-03-02 23:59:00.0,,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=1ea220bed10d4910VgnVCM2000001f4a900aRCRD,Biblioteca Pública Municipal Iván de Vargas (Centro),"{'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Centro'}, 'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Centro/Barrio/Palacio', 'locality': 'MADRID', 'postal-code': '28005', 'street-address': 'CALLE SAN JUSTO 5'}}","{'latitude': 40.41410290844665, 'longitude': -3.7098591212735883}","{'organization-name': 'Biblioteca Pública Municipal Iván de Vargas (Centro)', 'accesibility': '1,6'}",28005
1,12651622,3CLONWS,2025-03-21 19:00:00.0,2025-03-22 23:59:00.0,19:00,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=fc8d040e7c784910VgnVCM1000001d4a900aRCRD,Teatro Circo Price,"{'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Arganzuela'}, 'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Arganzuela/Barrio/PalosFrontera', 'locality': 'MADRID', 'postal-code': '28012', 'street-address': 'RONDA ATOCHA 35'}}","{'latitude': 40.40596936645757, 'longitude': -3.698589986849812}","{'organization-name': 'Teatro Circo Price', 'accesibility': '1'}",28012
2,12693864,60 Premio Reina Sofía de Pintura y Escultura,2025-02-27 00:00:00.0,2025-03-23 23:59:00.0,,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=34c5130393e15910VgnVCM2000001f4a900aRCRD,Centro Cultural Casa de Vacas (Retiro),"{'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Retiro'}, 'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Retiro/Barrio/LosJeronimos', 'locality': 'MADRID', 'postal-code': '28009', 'street-address': 'PASEO COLOMBIA 1'}}","{'latitude': 40.41906571939961, 'longitude': -3.684053112760816}","{'organization-name': 'Centro Cultural Casa de Vacas (Retiro)', 'accesibility': '1'}",28009
3,12691220,7 pares de pies,2025-03-23 12:00:00.0,2025-03-23 23:59:00.0,12:00,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=9d8ec08889415910VgnVCM2000001f4a900aRCRD,Centro Cultural Antonio Machado (San Blas - Canillejas),"{'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/SanBlas-Canillejas'}, 'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/SanBlas-Canillejas/Barrio/Arcos', 'locality': 'MADRID', 'postal-code': '28037', 'street-address': 'CALLE SAN ROMAN DEL VALLE 8'}}","{'latitude': 40.420280457570335, 'longitude': -3.621730741880216}","{'organization-name': 'Centro Cultural Antonio Machado (San Blas - Canillejas)', 'accesibility': '1'}",28037
4,12518159,A la fresca,2025-02-06 00:00:00.0,2025-02-23 23:59:00.0,,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=cd729f45bf9e1910VgnVCM1000001d4a900aRCRD,Nave 10 Matadero. Creación Dramática Contemporánea,"{'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Arganzuela'}, 'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Arganzuela/Barrio/Chopera', 'locality': 'MADRID', 'postal-code': '28045', 'street-address': 'PASEO CHOPERA 14'}}","{'latitude': 40.39245798191536, 'longitude': -3.6972067902718897}","{'organization-name': 'Nave 10 Matadero. Creación Dramática Contemporánea', 'accesibility': '1,6'}",28045
